# MAD1 family - scRNAseq from PBMCs

In [ ]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import scanpy as sc

In [ ]:
sc.logging.print_header()
sc.settings.verbosity = 3  

In [ ]:
# This does not work
sc.settings.set_figure_params(ipython_format="retina")

In [ ]:
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

#### August 18, 2021

## Read individual .h5 files from 10xGenomics

In [ ]:
# Hija
P = "/Users/mmm/BioPROJECTS/MAD1 & MVA/scRNAseq PBMCs/hija/"
ad_hija = sc.read_10x_h5(P + "filtered_feature_bc_matrix.h5")

In [ ]:
ad_hija.var[ad_hija.var.index.duplicated()]

In [ ]:
# using the default does not work: use join="_"
ad_hija.var_names_make_unique(join="_")

In [ ]:
ad_hija.var[ad_hija.var.index.duplicated()]

In [ ]:
ad_hija.var.index.is_unique

In [ ]:
ad_hija.var.loc["TBCE",:]

## Read the final files

In [ ]:
# Hija
P = "/Users/mmm/BioPROJECTS/MAD1 & MVA/scRNAseq PBMCs/data/original_from_10x/hija/"
ad_hija = sc.read_10x_h5(P + "filtered_feature_bc_matrix.h5")
ad_hija.var_names_make_unique(join="_")
ad_hija.obs["Sample"] = "Proband"
ad_hija

In [ ]:
# Madre
P = "/Users/mmm/BioPROJECTS/MAD1 & MVA/scRNAseq PBMCs/data/original_from_10x/madre/"
ad_madre = sc.read_10x_h5(P + "filtered_feature_bc_matrix.h5")
ad_madre.var_names_make_unique(join="_")
ad_madre.obs["Sample"] = "Mother"
ad_madre

In [ ]:
# Padre
P = "/Users/mmm/BioPROJECTS/MAD1 & MVA/scRNAseq PBMCs/data/original_from_10x/padre/"
ad_padre = sc.read_10x_h5(P + "filtered_feature_bc_matrix.h5")
ad_padre.var_names_make_unique(join="_")
ad_padre.obs["Sample"] = "Father"
ad_padre

In [ ]:
# C1
P = "/Users/mmm/BioPROJECTS/MAD1 & MVA/scRNAseq PBMCs/data/original_from_10x/C1/"
ad_C1 = sc.read_10x_h5(P + "filtered_feature_bc_matrix.h5")
ad_C1.var_names_make_unique(join="_")
ad_C1.obs["Sample"] = "Control1"
ad_C1

In [ ]:
# C2
P = "/Users/mmm/BioPROJECTS/MAD1 & MVA/scRNAseq PBMCs/data/original_from_10x/C2/"
ad_C2 = sc.read_10x_h5(P + "filtered_feature_bc_matrix.h5")
ad_C2.var_names_make_unique(join="_")
ad_C2.obs["Sample"] = "Control2"
ad_C2

### Concatenate

In [ ]:
adata = ad_hija.concatenate(ad_madre, ad_padre, ad_C1, ad_C2, )   #index_unique=None

In [ ]:
adata.obs

## Save .h5ad and .csv

In [ ]:
P = "/Users/mmm/BioPROJECTS/MAD1 & MVA/scRNAseq PBMCs/data/"
adata.write(P + "210818_MAD1_PBMCs.h5ad")

In [ ]:
adata.write_csvs(P + "210818_MAD1_PBMCs")

## Pre-Processing

In [ ]:
adata = sc.read_h5ad("/Users/mmm/BioPROJECTS/MAD1 & MVA/scRNAseq PBMCs/data/210818_MAD1_PBMCs.h5ad")

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20, )

In [ ]:
sc.pp.filter_cells(adata, min_genes=100)
sc.pp.filter_genes(adata, min_cells=3)

In [ ]:
# Mithochondrial 
adata.var['mt'] = adata.var_names.str.startswith('MT-')        # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

In [ ]:
# Remove cells that have too many mitochondrial genes expressed or too many total counts:
adata = adata[adata.obs.n_genes_by_counts < 4000, :]
adata = adata[adata.obs.pct_counts_mt < 50, :]

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)

In [ ]:
sc.pp.log1p(adata)

In [ ]:
# highly variable genes
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

In [ ]:
sc.pl.highly_variable_genes(adata)

In [ ]:
adata.raw = adata

#### Not done
If you don’t proceed below with correcting the data with sc.pp.regress_out and scaling it via sc.pp.scale, you can also get away without using .raw at all.

The result of the previous highly-variable-genes detection is stored as an annotation in .var.highly_variable and auto-detected by PCA and hence, sc.pp.neighbors and subsequent manifold/graph tools. In that case, the step actually do the filtering below is unnecessary, too.

Actually do the filtering

`adata = adata[:, adata.var.highly_variable]`

Regress out effects of total counts per cell and the percentage of mitochondrial genes expressed. Scale the data to unit variance.

`sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])`

regressing out ['total_counts', 'pct_counts_mt']
    sparse input is densified and may lead to high memory use
    finished (0:00:06)

Scale each gene to unit variance. Clip values exceeding standard deviation 10.

`sc.pp.scale(adata, max_value=10)`


## Principal component analysis

In [ ]:
sc.tl.pca(adata, svd_solver='arpack')

In [ ]:
sc.pl.pca(adata, color=['CST3', 'Sample'])

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True)

In [ ]:
# Save the .h5ad file
P = "/Users/mmm/BioPROJECTS/MAD1 & MVA/scRNAseq PBMCs/data/"
adata.write(P + "210818_MAD1_PBMCs.h5ad")

### UMAP

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color=['CST3', 'NKG7', 'PPBP'])

### Clustering the neighborhood graph

In [ ]:
sc.tl.leiden(adata, resolution=0.3)
sc.pl.umap(adata, color=['leiden', 'CST3', 'NKG7'])

In [ ]:
# Save the .h5ad file
P = "/Users/mmm/BioPROJECTS/MAD1 & MVA/scRNAseq PBMCs/data/"
adata.write(P + "210818_MAD1_PBMCs.h5ad")

In [ ]:
adata

In [ ]:
adata.write_csvs(P + "210818_MAD1_PBMCs")

In [ ]:
adata.obs.Sample.value_counts()

- 33604 cells
    - Proband:     6524
    - Mother:      6102
    - Father:      9046
    - Control1:    4522    
    - Control2:    7410
- 24339 genes

## Batch effect correction
Check whether it is really needed!!!

In [ ]:
adata = sc.read_h5ad("/Users/mmm/BioPROJECTS/MAD1 & MVA/scRNAseq PBMCs/data/210818_MAD1_PBMCs.h5ad")

In [ ]:
sc.pl.umap(adata, color=['leiden', 'Sample'])

#### Possibilities: 
`sc.pp.regress_out()`, `sc.pp.combat()`, Scanorama (`scanorama.correct`), MNN_Correct (`sc.external.pp.mnn_correc`)
#### Use sc.pp.combat()
- Comparison of batch correction methods in Scanpy: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7338326/

In [ ]:
sc.pp.regress_out(adata, keys="batch")            # default in sc.pp.combat key="batch"

In [ ]:
sc.pp.scale(adata, max_value=10)

### PCA and UMAP

In [ ]:
sc.tl.pca(adata, svd_solver='arpack')

In [ ]:
sc.pl.pca(adata, color=['CST3', 'Sample'])

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color=['leiden', 'Sample'])